<a href="https://colab.research.google.com/github/gonida1010/Fitogether-Soccer-Analytics-YOLOv8/blob/main/%EC%B6%95%EA%B5%AC%EC%9E%A5_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/00_자습 및 메모/과제/data/fittogether_org_data/fittogether_data

/content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/00_자습 및 메모/과제/data/fittogether_org_data/fittogether_data


In [ ]:
!unzip -q fittogether_data.zip

In [1]:
!pip install ultralytics opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.1 MB/s eta 0:00:00


In [2]:
import os
import random
import shutil
import cv2
import glob
import json
import yaml
import ultralytics
import matplotlib.pyplot as plt
from torchvision import transforms
from tqdm import tqdm
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
ultralytics.checks()

Ultralytics 8.3.249 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 45.7/112.6 GB disk)


In [ ]:
# 코랩 환경이 너무 오래 걸려서 원본 파일 다운과 압축 풀기는 코랩 -> txt 변환은 로컬에서 했다.

In [ ]:
data_root = '/content'
pjt_name = 'SelectStar_Fitogether'
pjt_root = f'{data_root}/{pjt_name}'

train_root = '/content/train'
valid_root = '/content/valid'
test_root = '/content/test'

for folder in [train_root, valid_root, test_root]:
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    for s in ['images', 'labels']:
        s_folder = os.path.join(folder, s)
        if not os.path.exists(s_folder):
            os.makedirs(s_folder, exist_ok=True)

print(f"프로젝트 루트: {pjt_root}")
print(f"학습 데이터 경로: {train_root}")

프로젝트 루트: /content/SelectStar_Fitogether
학습 데이터 경로: /content/train


In [ ]:
class_map = {"players": 0, "ball": 1, "others": 2}

def json_to_yolo_bbox(points, w, h):
    x_coords = [p[0] for p in points]
    y_coords = [p[1] for p in points]

    xmin, xmax = min(x_coords), max(x_coords)
    ymin, ymax = min(y_coords), max(y_coords)

    x_center = ((xmin + xmax) / 2) / w
    y_center = ((ymin + ymax) / 2) / h
    width = abs(xmax - xmin) / w
    height = abs(ymax - ymin) / h

    return [x_center, y_center, width, height]

In [ ]:
file_list = glob.glob(f'{data_root}/*.json')

In [ ]:
for file in tqdm(file_list, desc="Processing Fitogether Data"):
    result = []
    with open(file, 'r') as f:
        json_data = json.load(f)
        width = json_data['imageWidth']
        height = json_data['imageHeight']

        for shape in json_data['shapes']:
            label = shape['label']
            if label in class_map:
                cls = class_map[label]
                yolo_bbox = json_to_yolo_bbox(shape['points'], width, height)
                bbox_string = " ".join([str(x) for x in yolo_bbox])
                result.append(f'{cls} {bbox_string}')

    if result:
        with open(file.replace('.json', '.txt'), 'w', encoding='utf-8') as t:
            t.write('\n'.join(result))

In [3]:
data_path = '/content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/00_자습 및 메모/과제/data/fittogether_org_data'
os.chdir(data_path)

In [4]:
!unzip -q train.zip -d /content/
!unzip -q valid.zip -d /content/
!unzip -q test.zip -d /content/

In [ ]:
random.seed(2026)
file_list = glob.glob(f'{data_root}/*.txt')
random.shuffle(file_list)

test_ratio = 0.1
num_file = len(file_list)

test_list = file_list[:int(num_file*test_ratio)]
valid_list = file_list[int(num_file*test_ratio):int(num_file*test_ratio)*2]
train_list = file_list[int(num_file*test_ratio)*2:]

In [ ]:
def copy_files(target_list, target_root):
    for i in tqdm(target_list):
        txt_name = i.split('/')[-1]
        shutil.copyfile(i, f'{target_root}/labels/{txt_name}')

        img_path = i.replace('.txt', '.jpg')
        img_name = img_path.split('/')[-1]
        if os.path.exists(img_path):
            shutil.copyfile(img_path, f'{target_root}/images/{img_name}')

In [ ]:
copy_files(test_list, test_root)
copy_files(valid_list, valid_root)
copy_files(train_list, train_root)

100%|██████████| 188/188 [00:13<00:00, 13.93it/s]


In [ ]:
data_yaml = {
    'train': '/content/train/images',
    'val': '/content/valid/images',
    'test': '/content/test/images',
    'nc': 3,
    'names': ['players', 'ball', 'others']
}

with open('/content/fitogether.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

In [ ]:
# 코랩 T4 사용
model_n = YOLO('yolov8n.pt')
results_n = model_n.train(
    data='/content/fitogether.yaml',
    epochs=30,
    imgsz=416,
    batch=16,
    device=0, workers=3, amp=False,
    name='fit_nano_results'
)

Ultralytics 8.3.249 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/fitogether.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fit_nano_results2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

In [ ]:
# 로컬 GPU RTX4070TI 사용

model_s = YOLO('yolov8s.pt')
results_s = model_s.train(
    data='C:\Pyg\Fitogether-Soccer-Analytics-YOLOv8',
    epochs=40,
    imgsz=640,
    batch=64,
    device=0, workers=6, amp=True,
    name='fit_small_results'
)

In [5]:
# 1. 실제 데이터가 들어있는 구글 드라이브의 상위 경로
dataset_root = '/content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/00_자습 및 메모/과제/data'

model_n = YOLO(os.path.join('/content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/00_자습 및 메모/과제/data/fit_nano_results/weights/best.pt'))
model_s = YOLO(os.path.join('/content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/00_자습 및 메모/과제/data/fit_small_results/weights/best.pt'))

metrics_n = model_n.val(data='/content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/00_자습 및 메모/과제/data/fittogether_org_data/fitogether.yaml')
metrics_s = model_s.val(data='/content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/00_자습 및 메모/과제/data/fittogether_org_data/local_fitogether.yaml')

Ultralytics 8.3.250 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3954.1±1229.5 MB/s, size: 326.1 KB)
val: Scanning /content/valid/labels... 1114 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1114/1114 1.4Kit/s 0.8s
val: New cache created: /content/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 70/70 7.2it/s 9.7s
                   all       1114      20484      0.924      0.478      0.559      0.308
               players       1114      17529       0.95      0.737      0.896      0.512
                  ball        623        624          1          0          0          0
                others       1114       2331      0.821      0.697      0.779      0.412
Speed: 0.3ms preprocess, 2.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved